In [ ]:
import scipy.stats as sp
import numpy as np

In [ ]:
normal = sp.distributions.norm(loc=0, scale=1)

In [ ]:
samples = normal.rvs(10)
samples

In [ ]:
# for light tailed distr
from typing import Tuple
SMALL_VALUE: int = - 1e4
LARGE_VALUE: int = 1e4

def sign(x): return 1 if x >= 0 else -1

def interpolate(x_min, x_max, n) -> list:
    return np.linspace(x_min, x_max, n).tolist()

def find_min_max(distr, thresh: float) -> Tuple[float, float]:
    bd = (LARGE_VALUE + SMALL_VALUE) / 2
    found_min = False
    found_max = False
    x_min = None
    x_max = None

    while not (found_min):

        print(bd)
        if abs(distr(bd) - thresh) < 1e-5: 
            x_min = bd
            found_min = True
        else:
            # too far away -> adjust
            bd += (distr(bd) - thresh) * (bd + SMALL_VALUE)
        
    # reset, start search again for max val
    bd = (LARGE_VALUE + SMALL_VALUE) / 2
    while not (found_max):
        print(bd)
        if abs(distr(bd) - (1 - thresh)) < 1e-5: 
            x_max = bd
            found_max = True
        else:
            bd -= (distr(bd) - (1-thresh)) * (bd + LARGE_VALUE)
    return x_min, x_max
    

def approx(n: int, distr: callable):
    # x_min, x_max = find_min_max(distr)
    x_min, x_max = -20, 20
    # interpolate
    xs = interpolate(x_min, x_max, n)
    shift_left = [SMALL_VALUE]
    shift_left.extend(xs)
    xs.extend([LARGE_VALUE])
    

    # determine weights for bins
    bins = [(x, y) for x,y in zip(shift_left, xs)]
    mid_points = [b[1] - b[0] for b in bins]
    probs = [(distr(b[1]) - distr(b[0])) for b in bins]
    return probs 




In [ ]:
normal.cdf(-20)
# find_min_max(normal.cdf, 0.05)
temp = approx(100, normal.cdf)
# ls = interpolate(-20,20,10)
np.sum(temp)
np.count_nonzero(np.where(np.asarray(temp) >= 0))

# In theory, this works, needs work for find_min_max in a reliable way